In [ ]:
import warnings
warnings.filterwarnings("ignore")
import streamlit as st
import pandas as pd
import json
import re
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.snowpark import types as T
from snowflake.snowpark import Row
from snowflake.core import Root
from snowflake.cortex import Complete
session = get_active_session()

In [ ]:
-- Create a dedicated role & service user
USE ROLE ACCOUNTADMIN;

-- Grant privileges to the AICOLLEGE role
GRANT USAGE, OPERATE ON WAREHOUSE snowflake_intelligence_wh TO ROLE snowflake_intelligence_admin_rl;
GRANT ALL ON DATABASE si_hr_demo TO ROLE snowflake_intelligence_admin_rl;
GRANT ALL ON SCHEMA si_hr_demo.public TO ROLE snowflake_intelligence_admin_rl;
GRANT CREATE STAGE ON SCHEMA si_hr_demo.public TO ROLE snowflake_intelligence_admin_rl;
GRANT SELECT ON FUTURE TABLES IN SCHEMA si_hr_demo.public TO ROLE snowflake_intelligence_admin_rl;
GRANT SELECT ON FUTURE VIEWS  IN SCHEMA si_hr_demo.public TO ROLE snowflake_intelligence_admin_rl;
GRANT READ ON STAGE si_hr_demo.public.hr_docs TO ROLE snowflake_intelligence_admin_rl;
GRANT DATABASE ROLE snowflake.cortex_user TO ROLE snowflake_intelligence_admin_rl;

In [ ]:
-- View the list of files in the stage to ensure 6 are present
LIST @SI_HR_DEMO.PUBLIC.HR_DOCS PATTERN='.*\.pdf$';

In [ ]:
-- View an HR doc
SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    '@SI_HR_DEMO.PUBLIC.HR_DOCS',
    'international_travel_policy.pdf',
    OBJECT_CONSTRUCT('mode','layout')
  ) AS layout;

In [ ]:
USE ROLE snowflake_intelligence_admin_rl;

-- Create the parsed_docs table
CREATE TABLE IF NOT EXISTS PARSED_HR_DOCS (
    RELATIVE_PATH  STRING PRIMARY KEY,          -- full path in the stage
    RAW_TEXT       VARIANT,                     -- PDF text
    LOADED_AT      TIMESTAMP_TZ DEFAULT CURRENT_TIMESTAMP()
);

-- Process files one at a time to avoid stage scanning issues
INSERT INTO PARSED_HR_DOCS (RELATIVE_PATH, RAW_TEXT)
SELECT 
    RELATIVE_PATH,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(      --> Use Snowflake's Parse Document function
        '@SI_HR_DEMO.PUBLIC.HR_DOCS',
        RELATIVE_PATH,
        OBJECT_CONSTRUCT('mode','layout') --> Use layout mode
    ) AS RAW_TEXT
FROM DIRECTORY('@SI_HR_DEMO.PUBLIC.HR_DOCS') f
WHERE
  RELATIVE_PATH NOT IN (SELECT RELATIVE_PATH FROM PARSED_HR_DOCS);

In [ ]:
-- Validate PARSED_DOCS was created correctly
SELECT * FROM PARSED_HR_DOCS LIMIT 5;

In [ ]:
-- Create chunks of the parsed transcripts
CREATE TABLE IF NOT EXISTS HR_DOC_CHUNKS (
    RELATIVE_PATH  STRING,
    CHUNK          STRING
);

-- Insert chunked content using SPLIT_TEXT_RECURSIVE_CHARACTER
INSERT INTO HR_DOC_CHUNKS (RELATIVE_PATH, CHUNK)
WITH text_chunks AS (
    SELECT
        RELATIVE_PATH,
        SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(  --> Use Snowflake's Split Text Recursive function
            RAW_TEXT:content::STRING,  -- extract string content
            'markdown',                -- tokenizer
            800,                       -- chunk size
            100,                       -- overlap
            ARRAY_CONSTRUCT('\n\n')    -- preferred break
        ) AS CHUNKS
    FROM PARSED_HR_DOCS
    WHERE RAW_TEXT:content IS NOT NULL -- optional safety check
)
SELECT
    RELATIVE_PATH,
    chunk.value::STRING AS CHUNK  --> Save chunks of parsed transcripts as "CHUNK"
FROM text_chunks,
LATERAL FLATTEN(input => CHUNKS) AS chunk;  --> Flatten the chuncked output

In [ ]:
-- Validate HR_DOC_CHUNKS was created correctly
SELECT * FROM HR_DOC_CHUNKS LIMIT 5;

In [ ]:
-- Create a Cortex Search Service over your chunked HR docs table
CREATE OR REPLACE CORTEX SEARCH SERVICE SI_HR_DEMO.PUBLIC.HR_DOCS_SEARCH_SERVICE 
    ON CHUNK
    ATTRIBUTES RELATIVE_PATH
    WAREHOUSE = SNOWFLAKE_INTELLIGENCE_WH
    TARGET_LAG = '365 days'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'  -- Optional: Select your desired embedding model
    AS (
    SELECT
        RELATIVE_PATH,
        CHUNK::VARCHAR AS CHUNK,
    FROM HR_DOC_CHUNKS);

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE EMPLOYEE_NAME_SEARCH
  ON full_name
  ATTRIBUTES employee_id, first_name, last_name
  WAREHOUSE = SNOWFLAKE_INTELLIGENCE_WH
  TARGET_LAG = '24 hour'
  AS (
      SELECT 
          EMPLOYEE_ID,
          FIRST_NAME,
          LAST_NAME,
          FIRST_NAME || ' ' || LAST_NAME AS full_name
      FROM SI_HR_DEMO.PUBLIC.EMPLOYEES
  );

In [ ]:
SELECT * FROM EMPLOYEES